In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/Dataset.zip"  # Path to the copied ZIP file
extract_path = "/content/extracted"  # Destination folder

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction completed. Files are in:", extract_path)


Extraction completed. Files are in: /content/extracted


In [ ]:
import cv2
import mediapipe as mp
import csv
import copy
import itertools
import os

# Initialize Mediapipe Hand Module
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Function to extract hand landmarks (x, y, z)
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]
    return [
        [min(int(l.x * image_width), image_width - 1),
         min(int(l.y * image_height), image_height - 1),
         l.z]  # Depth (relative to wrist)
        for l in landmarks.landmark
    ]

# Function to normalize landmarks (x, y, z)
def pre_process_landmark(landmark_list):
    base_x, base_y, base_z = landmark_list[0]  # Use wrist as reference
    temp_landmark_list = [[x - base_x, y - base_y, z - base_z] for x, y, z in landmark_list]

    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list))  # Flatten
    max_value = max(map(abs, temp_landmark_list), default=1)  # Normalize
    return [n / max_value for n in temp_landmark_list]

# Function to log data into CSV
def logging_csv(letter, landmark_list):
    csv_path = '/content/keypointAug.csv'  # Save CSV in Colab workspace
    with open(csv_path, 'a', newline="") as f:
        writer = csv.writer(f)
        writer.writerow([letter, *landmark_list])  # Save label + features

# **Define dataset path**
dataset_path = '/content/extracted/Dataset'

# **Dynamically get all class labels (folder names)**
if not os.path.exists(dataset_path):
    print(f"❌ Dataset directory not found: {dataset_path}")
    exit()

labels = sorted([f for f in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, f))])
print(f"📂 Found {len(labels)} classes: {labels}")

# Initialize Mediapipe Hands Model
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:

    # **Loop through each folder (class label)**
    for label in labels:
        folder_path = os.path.join(dataset_path, label)
        image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

        print(f"🔄 Processing class: {label} ({len(image_files)} images)")

        # **Loop through each image**
        for file in image_files:
            if not os.path.exists(file):
                print(f"❌ File not found: {file}")
                continue

            # **Read Image**
            image = cv2.imread(file)
            if image is None:
                print(f"⚠️ Corrupt image: {file}")
                continue

            image = cv2.flip(image, 1)  # Flip for consistency
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            if not results.multi_hand_landmarks:
                print(f"🚫 No hand detected in {file}")
                continue

            # **Process landmarks**
            for hand_landmarks in results.multi_hand_landmarks:
                landmark_list = calc_landmark_list(image, hand_landmarks)
                pre_processed_landmark_list = pre_process_landmark(landmark_list)

                # **Debug output**
                print(f"✅ Processed: {file} -> Label: {label}")

                # **Log to CSV**
                logging_csv(label, pre_processed_landmark_list)

print("✅ Data extraction complete! CSV saved at /content/keypoint.csv")
